In [1]:
import pandas as pd
from sklearn import tree
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('../data/Cars93.csv')

In [3]:
df = df[df['Cylinders'] != 'rotary']

In [4]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
#shows which values may heavily correlate with each other (collinearity)
XX = df._get_numeric_data()
for col in XX:
    if XX[col].isnull().sum() > 0:
        XX[col].fillna(XX[col].mean(), inplace = True)
vif_data = pd.DataFrame()
vif_data["feature"] = XX.columns
vif_data['VIF'] = [variance_inflation_factor(XX.values, i) for i in range(len(XX.columns))]
vif_data_sorted = vif_data.sort_values(['VIF'])
print(vif_data_sorted)

               feature            VIF
0           Unnamed: 0       5.055568
17        Luggage.room      71.631555
9         Rev.per.mile     100.701697
11          Passengers     124.452168
7           Horsepower     151.622923
6           EngineSize     187.305998
10  Fuel.tank.capacity     197.987176
16      Rear.seat.room     269.105523
4             MPG.city     313.935329
8                  RPM     403.876339
5          MPG.highway     517.567116
15         Turn.circle     538.171902
18              Weight     717.452510
12              Length     976.174614
14               Width    1983.082955
13           Wheelbase    2219.712876
1            Min.Price  133353.564065
3            Max.Price  217735.917366
2                Price  684518.336074


In [5]:
df.isna().sum()

Unnamed: 0            0
Manufacturer          0
Model                 0
Type                  0
Min.Price             0
Price                 0
Max.Price             0
MPG.city              0
MPG.highway           0
AirBags               0
DriveTrain            0
Cylinders             0
EngineSize            0
Horsepower            0
RPM                   0
Rev.per.mile          0
Man.trans.avail       0
Fuel.tank.capacity    0
Passengers            0
Length                0
Wheelbase             0
Width                 0
Turn.circle           0
Rear.seat.room        0
Luggage.room          0
Weight                0
Origin                0
Make                  0
dtype: int64

#Practice

In [6]:
df.columns

Index(['Unnamed: 0', 'Manufacturer', 'Model', 'Type', 'Min.Price', 'Price',
       'Max.Price', 'MPG.city', 'MPG.highway', 'AirBags', 'DriveTrain',
       'Cylinders', 'EngineSize', 'Horsepower', 'RPM', 'Rev.per.mile',
       'Man.trans.avail', 'Fuel.tank.capacity', 'Passengers', 'Length',
       'Wheelbase', 'Width', 'Turn.circle', 'Rear.seat.room', 'Luggage.room',
       'Weight', 'Origin', 'Make'],
      dtype='object')

In [7]:
dummyVars = pd.get_dummies(df[['DriveTrain', 'Man.trans.avail', 'Origin', 'AirBags', 'Manufacturer', 'Cylinders','Type']])
dummyVars

,DriveTrain_4WD,DriveTrain_Front,DriveTrain_Rear,Man.trans.avail_No,Man.trans.avail_Yes,Origin_USA,Origin_non-USA,AirBags_Driver & Passenger,AirBags_Driver only,AirBags_None,Manufacturer_Acura,Manufacturer_Audi,Manufacturer_BMW,Manufacturer_Buick,Manufacturer_Cadillac,Manufacturer_Chevrolet,Manufacturer_Chrylser,Manufacturer_Chrysler,Manufacturer_Dodge,Manufacturer_Eagle,Manufacturer_Ford,Manufacturer_Geo,Manufacturer_Honda,Manufacturer_Hyundai,Manufacturer_Infiniti,Manufacturer_Lexus,Manufacturer_Lincoln,Manufacturer_Mazda,Manufacturer_Mercedes-Benz,Manufacturer_Mercury,Manufacturer_Mitsubishi,Manufacturer_Nissan,Manufacturer_Oldsmobile,Manufacturer_Plymouth,Manufacturer_Pontiac,Manufacturer_Saab,Manufacturer_Saturn,Manufacturer_Subaru,Manufacturer_Suzuki,Manufacturer_Toyota,Manufacturer_Volkswagen,Manufacturer_Volvo,Cylinders_3,Cylinders_4,Cylinders_5,Cylinders_6,Cylinders_8,Type_Compact,Type_Large,Type_Midsize,Type_Small,Type_Sporty,Type_Van
0,0,1,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
1,0,1,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
2,0,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
3,0,1,0,0,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
4,0,0,1,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1
89,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0
90,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0
91,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0


In [8]:
df.drop(['Min.Price', 'Max.Price', 'MPG.city', 'MPG.highway','Rev.per.mile','DriveTrain', 'Man.trans.avail', 'Origin', 'AirBags', 'Manufacturer', 'Cylinders','Type','Model','Make', 'Unnamed: 0'],inplace =True,axis='columns')

In [9]:
df = pd.concat([df, dummyVars],axis=1)
df

,Price,EngineSize,Horsepower,RPM,Fuel.tank.capacity,Passengers,Length,Wheelbase,Width,Turn.circle,Rear.seat.room,Luggage.room,Weight,DriveTrain_4WD,DriveTrain_Front,DriveTrain_Rear,Man.trans.avail_No,Man.trans.avail_Yes,Origin_USA,Origin_non-USA,AirBags_Driver & Passenger,AirBags_Driver only,AirBags_None,Manufacturer_Acura,Manufacturer_Audi,Manufacturer_BMW,Manufacturer_Buick,Manufacturer_Cadillac,Manufacturer_Chevrolet,Manufacturer_Chrylser,Manufacturer_Chrysler,Manufacturer_Dodge,Manufacturer_Eagle,Manufacturer_Ford,Manufacturer_Geo,Manufacturer_Honda,Manufacturer_Hyundai,Manufacturer_Infiniti,Manufacturer_Lexus,Manufacturer_Lincoln,Manufacturer_Mazda,Manufacturer_Mercedes-Benz,Manufacturer_Mercury,Manufacturer_Mitsubishi,Manufacturer_Nissan,Manufacturer_Oldsmobile,Manufacturer_Plymouth,Manufacturer_Pontiac,Manufacturer_Saab,Manufacturer_Saturn,Manufacturer_Subaru,Manufacturer_Suzuki,Manufacturer_Toyota,Manufacturer_Volkswagen,Manufacturer_Volvo,Cylinders_3,Cylinders_4,Cylinders_5,Cylinders_6,Cylinders_8,Type_Compact,Type_Large,Type_Midsize,Type_Small,Type_Sporty,Type_Van
0,15.9,1.8,140,6300,13.2,5,177,102,68,37,26.5,11.000000,2705,0,1,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
1,33.9,3.2,200,5500,18.0,5,195,115,71,38,30.0,15.000000,3560,0,1,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
2,29.1,2.8,172,5500,16.9,5,180,102,67,37,28.0,14.000000,3375,0,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
3,37.7,2.8,172,5500,21.1,6,193,106,70,37,31.0,17.000000,3405,0,1,0,0,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
4,30.0,3.5,208,5700,21.1,4,186,109,69,39,27.0,13.000000,3640,0,0,1,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,19.7,2.5,109,4500,21.1,7,187,115,72,38,34.0,13.890244,3960,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1
89,20.0,2.0,134,5800,18.5,5,180,103,67,35,31.5,14.000000,2985,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0
90,23.3,2.8,178,5800,18.5,4,159,97,66,36,26.0,15.000000,2810,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0
91,22.7,2.3,114,5400,15.8,5,190,104,67,37,29.5,14.000000,2985,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0


In [10]:
#what do we want to predict
y = df['Wheelbase']


In [11]:
df.drop(['Wheelbase'], inplace = True, axis='columns')


In [12]:
X = df.values

In [13]:
dtree = tree.DecisionTreeRegressor()

In [14]:
#X_train, X_test, y_train, y_test = train_test_split(X, y)

In [15]:
results = train_test_split(X, y)

In [16]:
X_train, X_test, y_train, y_test = results

In [17]:
dtree.fit(X_train,y_train)

DecisionTreeRegressor()

In [18]:
dtree.score(X,y)

0.8669008254872077

In [19]:
dtree.predict(X_test)

array([102.,  97., 105., 116.,  93.,  97., 103.,  90.,  90., 102., 110.,
       110.,  90., 111., 107., 103., 113., 110., 103., 103., 103., 108.,
        98.])